# Simulation of graphene on QuEra

### TO DO:

#### general
- AWS access

#### problem-specific
- think of positive couplings
- periodic boudnary conditions
    - post-processing
    - "protective layer of atoms" (needs local fields)
    - scale up approach
- redefine the problem in terms of graph partitioning

## Build the graphene supercell

In [1]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import copy
import numpy as np

lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
structure = graphene_supercell
graphene_supercell.num_sites

18

graphene_supercell is a pymatgen Structure object

## Build the adjacency matrix

In [3]:
import numpy as np
    
distance_matrix_pbc = np.round(structure.distance_matrix,5)

shells = np.unique(distance_matrix_pbc[0])

adjacency_matrix = np.round(distance_matrix_pbc,5) == np.round(shells[1],5)
adjacency_matrix = adjacency_matrix.astype(int)
adjacency_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,

## The initial QUBO matrix

The initial QUBO matrix is built using the following assumptions:
- each bond decreases the energy of the system by 1 arbitrary unit
- only nearest neighbours are included in this model
- if a an atom is next to a vacancy or if two vacancies are next to each other there is no bond

The top right part of adjacency_matrix (changed by sign) can be used.

In [13]:
Q = -np.triu(adjacency_matrix)
Q

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0, -1,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0,
        -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,  0,
         0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1, -1,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1, -1,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,
        -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,
         0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  

## Calculating the energy classically using the QUBO matrix
The energy can be calculated by using:
$$
E(\textbf{x}) =  \textbf{x}^\textbf{T}\textbf{Qx} = 
    \sum_{i} Q_{i,i} x_{i} + \sum_{i} \sum_{j > i} Q_{i,j} x_{i}x_{j} \ \ \ \ \ x_{i} \in \{0,1\}
$$

In the $\textbf{x}$ vector, $x_i$= 1 corresponds to an atom in position $i$, $x_i$= 0 corresponds to a vacancy in position $i$

## 0 vacancy

In [17]:
x = np.array([1]*18)
Qx = np.matmul(x,Q)
E_0v = np.matmul(x,Qx)
E_0v

-27

## 1 vacancy

3 broken bonds

In [18]:
x = np.array([1]*18)
x[0] = 0
Qx = np.matmul(x,Q)
np.matmul(x,Qx)

-24

## 2 vacancies

Non-first neighbours (6 broken bonds)

In [19]:
x = np.array([1]*18)
x[0] = 0
x[1] = 0
Qx = np.matmul(x,Q)
np.matmul(x,Qx)

-21

First neighbours (5 broken bonds)

In [20]:
x = np.array([1]*18)
x[0] = 0
x[9] = 0
Qx = np.matmul(x,Q)
np.matmul(x,Qx)

-22

## Make interaction positive
Q_p = QUBO positive

In [35]:
Q_p = np.triu(adjacency_matrix)

In [33]:
x = np.array([1]*18)
Qx = np.matmul(x,Q_p)
np.matmul(x,Qx)

27

## 0 vacancy

In [34]:
x = np.array([1]*18)
Qx = np.matmul(x,Q_p)
E_0v = np.matmul(x,Qx)
E_0v

27

## 1 vacancy

3 broken bonds

In [26]:
x = np.array([0]*18)
x[0] = 1
Qx = np.matmul(x,Q_p)
np.matmul(x,Qx)

0

## 2 vacancies

Non-first neighbours (6 broken bonds)

In [19]:
x = np.array([1]*18)
x[0] = 0
x[1] = 0
Qx = np.matmul(x,Q)
np.matmul(x,Qx)

-21